In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import openpyxl  # openpyxl 모듈 추가
import re

sns.set_style('darkgrid')
sns.set_palette('Set3')
warnings.filterwarnings('ignore')
plt.rcParams["font.family"] = "NanumBarunGothic"


In [2]:
import mysql.connector

# 데이터베이스 연결
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='music',
    database='MusicDatabase'
)

cursor = conn.cursor()

# 쿼리 실행  기존에 존재하는 데이터는 제외하고  출력
# 각 플랫폼별 최신일자 Top10에 대한 정보 출력 ※idol_Group테이블에는 Top10에 대한 그룹정보 저장 
# 각 플랫폼별 최신일자 Top10에 대한 싱어코드 추출하여 싱어코드를 바탕으로 멜론 사이트에서 그룹정보 크롤링
query = """
    SELECT Singer_Code
    FROM Music
    WHERE Day = (SELECT max(Day) FROM Music)
    AND Ranking < 11
    AND Singer_Code not in (select Group_Code from idol_Group group by Group_Code)
    AND Singer_Code IS NOT NULL
    GROUP BY Singer_Code
    ORDER BY MIN(Ranking) ASC;
"""
cursor.execute(query)

# 결과 출력
Singer_Code_List = cursor.fetchall()
for (Singer_Code) in Singer_Code_List:
    print(Singer_Code)

# 커서와 연결 닫기
cursor.close()
conn.close()



In [3]:

Singer_Code_List

# 튜플의 첫 번째 요소만 추출
singer_codes = [code[0] for code in Singer_Code_List]

print(singer_codes)


[]


In [4]:
import requests
from bs4 import BeautifulSoup

# User-Agent 헤더
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
# 아티스트 정보 URL
url = "https://www.melon.com/artist/detail.htm?artistId=614026"

response = requests.get("https://www.melon.com/artist/detail.htm?artistId=614026")


print(url)
html = response.content.decode('utf-8', 'replace')
soup = BeautifulSoup(html, 'html.parser', from_encoding='cp949')

my_st = soup.select('#conts')

print(my_st)

for stock_name in my_st:

    # 이미지 URL 추출
    img_tag = soup.select_one('#conts > div.wrap_dtl_atist > div > div.wrap_thumb > span > span.thumb_frame')
    img_url = img_tag['src'] if img_tag else 'None'

    # 결과 출력
    print(f"Group_ImgUrl = {img_url}")
else:
    print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")


https://www.melon.com/artist/detail.htm?artistId=614026
[]
Failed to retrieve data. HTTP Status code: 406


In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Selenium 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저를 화면에 표시하지 않음

# Chrome 드라이버 경로 설정
driver_service = Service("C:/P_Project/chromedriver-win64/chromedriver.exe")  # 드라이버 경로 설정
driver = webdriver.Chrome(service=driver_service, options=chrome_options)

# URL 설정
url = "https://www.melon.com/artist/detail.htm?artistId=614026"
driver.get(url)

# 페이지가 로드될 때까지 대기
try:
    # 15초까지 대기하고 아티스트 이미지가 로드될 때까지 대기
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.wrap_thumb img'))
    )
except Exception as e:
    print(f"Error while waiting for the image: {e}")

# 페이지 소스 가져오기
html = driver.page_source

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(html, 'html.parser')

# 이미지 URL 추출
img_tag = soup.select_one('.wrap_thumb img')
img_url = img_tag['src'] if img_tag else 'None'

# 결과 출력
print(f"Group_ImgUrl = {img_url}")

# 브라우저 종료
driver.quit()


In [8]:
## DB에 있는 IMG와 멜론에 등록된 앨범IMG의 URL를 비교함

import mysql.connector

# 데이터베이스 연결
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='music',
    database='MusicDatabase'
)

cursor = conn.cursor()


# 최신일자 Top10에 대한 싱어코드를 추출하여 DB에 등록된 앨범이미지의 URL과 멜론 사이트의 앨범이미지 URL을 비교
query = """
    select Singer_Code From Music where 1=1 and ranking < 11 and day = (select Max(day) from Music) and Singer_Code is not null group by Singer_Code;
"""
cursor.execute(query)

# 결과 출력
Group_Code_List = cursor.fetchall()
for (Group_Code) in Group_Code_List:
    print(Group_Code)

# 커서와 연결 닫기
cursor.close()
conn.close()



(894864,)
(3610529,)
(2899555,)
(2019210,)
(3752136,)
(3114174,)


In [9]:
Group_Code_List

# 튜플의 첫 번째 요소만 추출
group_Codes = [code[0] for code in Group_Code_List]

print(group_Codes)


[894864, 3610529, 2899555, 2019210, 3752136, 3114174]


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import mysql.connector

# ChromeDriver 경로 설정
driver_path = 'C:\\P_Project\\chromedriver-win64\\chromedriver.exe'
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# MySQL 연결 설정
db = mysql.connector.connect(
    host='localhost',
    user='root',
    password='music',
    database='MusicDatabase'
)
cursor = db.cursor()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

for i in group_Codes:
    # 1. 크롤링: 그룹 코드에 대해 이미지 URL을 가져옴
    URL = f"https://www.melon.com/artist/timeline.htm?artistId={i}"
    driver.get(URL)
    time.sleep(1)  # 페이지 로딩 대기

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 이미지 URL 추출
    group_img_url = soup.select_one('span#artistImgArea img')['src'] if soup.select_one('span#artistImgArea img') else ''

    # 2. 위에서 싱어코드를 추출하여 DB에 등록된 앨범이미지의 URL과 멜론 사이트의 앨범이미지 URL을 비교
    query = f"SELECT A.Group_ImgUrl FROM idol_Group A, Music B WHERE A.Group_Code = {i} and A.Group_Name = B.Singer and B.Ranking < 11 Group by A.Group_ImgUrl"
    cursor.execute(query)
    db_img_url = cursor.fetchone()

    # 3. 비교 및 업데이트
    if db_img_url:
        db_img_url = db_img_url[0]  # fetchone은 튜플로 반환되므로 첫 번째 값 가져오기
        if group_img_url != db_img_url:  # 이미지 URL이 다를 경우
            update_query = f"UPDATE idol_Group SET Group_ImgUrl = %s WHERE Group_Code = %s"
            cursor.execute(update_query, (group_img_url, i))
            db.commit()  # 업데이트 적용
            print(f"Group_Code {i}의 이미지 URL을 업데이트했습니다.")
            print(f"SQL Query: UPDATE idol_Group SET Group_ImgUrl = '{group_img_url}' WHERE Group_Code = {i};")
        else:
            print(f"Group_Code {i}의 이미지 URL이 동일합니다.")
    else:
        print(f"Group_Code {i}에 대한 정보가 없습니다.")

# 드라이버 종료 및 DB 연결 해제
driver.quit()
cursor.close()
db.close()

Group_Code 894864의 이미지 URL이 동일합니다.
Group_Code 3610529의 이미지 URL이 동일합니다.
Group_Code 2899555의 이미지 URL을 업데이트했습니다.
SQL Query: UPDATE idol_Group SET Group_ImgUrl = 'https://cdnimg.melon.co.kr/cm2/artistcrop/images/028/99/555/2899555_20241016140306_500.jpg?b77f88c284b4942f1e779097ff66199c/melon/resize/416/quality/80/optimize' WHERE Group_Code = 2899555;
Group_Code 2019210의 이미지 URL이 동일합니다.
Group_Code 3752136의 이미지 URL이 동일합니다.
Group_Code 3114174의 이미지 URL이 동일합니다.
